In [1]:
from allthethings import PyNetwork
from allthethings import PyBC_opt_dh
import numpy as np

In [2]:
fi = "indata/3pipes1.inp"     #location of .inp file
fc = "indata/3pipes1.config"  #location of .config file
mtype = 1                     #model used along network edges. 1 for Preissman Slot. 0 for uniform
n1 = PyNetwork(fi, fc, mtype) #a network object
print n1 
#n1.showLayout()               #print the layout of the network


Network at address 0x10400f400 with 4 nodes and 3 edges



In [3]:
#n1.showCurrentData()    
dt = n1.T/n1.M
n1.runForwardProblem(dt)  #solve up to time T

In [5]:
n1.showCurrentData()
n1.getAveGradH(200)
n1.cmax

At time t = 50.000000
Data from pipe 0
A           Q
0.047972    -0.000526
0.047735    -0.001553
0.047446    -0.002444
0.047443    -0.002980
0.048061    -0.002939
0.049450    -0.002207
0.051576    -0.000778
0.054316    0.001317
0.057541    0.004039
0.061148    0.007368
0.065063    0.011291
0.069238    0.015809
0.073638    0.020927
0.078241    0.026654
0.083029    0.033003
0.087991    0.039987
0.093118    0.047622
0.098403    0.055920
0.103841    0.064896
0.109464    0.074544
0.115936    0.084554
0.132235    0.088072
0.196108    0.025745
0.306258    -0.166202
0.403888    -0.357434
Data from pipe 1
A           Q
0.548813    -0.326850
0.491291    -0.516712
0.447818    -0.582449
0.446270    -0.582611
0.457986    -0.564105
0.471611    -0.539589
0.484875    -0.512170
0.497320    -0.482977
0.508848    -0.452588
0.519457    -0.421396
0.529187    -0.389718
0.538096    -0.357836
0.546250    -0.326008
0.553715    -0.294480
0.560550    -0.263488
0.566806    -0.233251
0.572523    -0.203963
0.577721

[12.0, 12.0, 12.0]

In [8]:
ndof = 16
wn = np.array([1,2])
x0 = .1*np.ones(ndof*len(wn))
opt1 = PyBC_opt_dh(fi, fc, ndof, x0, 1, wn)

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]


In [ ]:
opt1.solve()
opt1.x
opt1.r
opt1.f